# predecessor_dist

In [44]:
#Class Project는 불변
#Class Schedule은 변경

In [9]:
import os
import sys
import itertools
import pickle as pk
import pandas as pd
from copy import deepcopy
from collections import defaultdict
from collections import defaultdict, Counter
from object import  Activity, Grid, Project
from naviutil import NaviPath, NaviFunc, NaviIO
navipath = NaviPath()
navifunc = NaviFunc()
naviio = NaviIO()

In [10]:
import pandas as pd
from naviutil import NaviPath
navipath = NaviPath()

activity_pre_dist = pd.read_excel(navipath.activity_pre_dist)


### loading activity dist template and making dic

In [11]:
activity_pre_dist_dic = {}
for _, line in activity_pre_dist.iterrows():
    activity_pre_dist_dic[line['code']] = line['predecessor_dist']

print(activity_pre_dist_dic)

{'M00009': 10, 'M00001': 0, 'M00007': 0, 'T10010': 3, 'T10020': 3, 'T10030': 3, 'T10040': 3, 'T10050': 3, 'T10060': 3, 'T10070': 3, 'T10080': 3, 'T10090': 3, 'T30010': 1, 'T30020': 1, 'T30030': 1, 'T30031': 1, 'T30032': 1, 'T30033': 1, 'T30034': 1, 'T30035': 1, 'T30036': 1, 'T30040': 1, 'T30050': 1, 'T32010': 1, 'T32020': 1, 'T35010': 1, 'T35020': 1, 'T35030': 1, 'T35031': 1, 'T35032': 1, 'T35033': 1, 'T35034': 1, 'T35035': 1, 'T35036': 1, 'T35040': 1, 'T35050': 1, 'T37010': 1, 'W10010': 5, 'W10020': 3, 'W10030': 2, 'W10040': 1, 'W10050': 1, 'W10060': 1, 'W10070': 1, 'W11010': 5, 'W11020': 1, 'W11030': 1, 'W11040': 1, 'D10010': 2, 'D10020': 0, 'D10030': 0, 'D11010': 3, 'D12010': 3, 'D12020': 3, 'R10010': 5, 'R10020': 3, 'R11010': 3, 'R11020': 3, 'R11030': 3, 'R12010': 5, 'R12020': 1, 'F20010': 2, 'F20020': 3, 'F20030': 2, 'F20040': 2, 'F20050': 3, 'F20051': 0, 'F20060': 2, 'F20070': 3, 'S10010': 3, 'S10020': 2, 'S10030': 1, 'S10040': 2, 'S10050': 3, 'S10060': 3, 'S10061': 0, 'S10070': 

### Loading updated schedule

In [13]:
activity_book = naviio.import_activity_book()
fpath = 'D:/cns/navi-master/navi/schedule/schedule_N-05_structure_C-updated_I-062.xlsx'
schedule = naviio.xlsx2schedule(activity_book=activity_book, fpath=fpath)
schedule


FileNotFoundError: [Errno 2] No such file or directory: 'D:/cns/navi-master/navi/schedule/schedule_N-05_structure_C-updated_I-062.xlsx'

In [7]:
schedule['2_1_-1']
activity_pre_dist_dic['S10010']

NameError: name 'schedule' is not defined

### checking activity & dist 

In [20]:
schedule.keys()#첫날의 작업 중 첫번째다 치고


dict_keys(['2_1_-1', '2_2_-1', '2_3_-1', '3_2_2', '3_3_2', '2_3_2', '3_1_2', '3_1_-1', '3_2_-1', '3_3_-1', '2_1_1', '2_2_1', '2_3_1', '3_1_1', '3_2_1', '3_3_1', '2_1_2', '2_2_2'])

In [28]:
check_pre_dist_act = schedule('2_1_-1')
checked_dist = activity_pre_dist_dic[check_pre_dist_act] #첫날 첫번째 작업의 선행완료거리값
print(checked_dist)

TypeError: 'collections.defaultdict' object is not callable

### grid & activity

In [51]:
act_grids ={}
for location, activity in schedule.iterrows():
    if activity[0] == check_pre_dist_act: #확인된 작업과 동일한 그리드
        act_grids[activity["Unnamed: 0"]] = activity[0] #확인된 작업과 동일한 그리드 딕셔너리 생성
print(act_grids)

{'3_2_2': 'S10010', '3_3_2': 'S10010', '2_3_2': 'S10010', '3_1_2': 'S10010', '2_2_2': 'S10010'}


###   finding grids inside influence of act

In [76]:
for grid, activity in act_grids.items():
    x, y, z = grid.split("_")
    pre_dist1 = checked_dist
    pre_dist2 = checked_dist
    pre_dist_z1 = 1
    pre_dist_z2 = int(-1)
    x = int(x)
    y = int(y)
    z = int(z)
    pre_dist_location_x_list=[]
    pre_dist_location_y_list=[]
    pre_dist_location_z_list=[z]
    while pre_dist1 > 0:
        pred_dist_x1 = x + pre_dist1
        pre_dist_location_x_list.append(pred_dist_x1)
        pred_dist_y1 = y + pre_dist1
        pre_dist_location_y_list.append(pred_dist_y1)
        pre_dist1 = pre_dist1 - 1

    while pre_dist2 >= 0:
        pred_dist_x2 = x - pre_dist2
        pred_dist_y2 = y - pre_dist2
        if pred_dist_x2 > 0:
            pre_dist_location_x_list.append(pred_dist_x2)
        if pred_dist_y2 > 0:
            pre_dist_location_y_list.append(pred_dist_y2)
        pre_dist2 = pre_dist2 - 1
    
    if 'D' in check_pre_dist_act:
        pred_dist_z = z + pre_dist_z1
        pre_dist_location_z_list.append(pred_dist_z)
    elif 'S' in check_pre_dist_act:
        pred_dist_z = z + pre_dist_z2
        pre_dist_location_z_list.append(pred_dist_z)
        
   
    print(pre_dist_location_x_list,pre_dist_location_y_list,pre_dist_location_z_list)


[6, 5, 4, 1, 2, 3] [5, 4, 3, 1, 2] [2, 1]
[6, 5, 4, 1, 2, 3] [6, 5, 4, 1, 2, 3] [2, 1]
[5, 4, 3, 1, 2] [6, 5, 4, 1, 2, 3] [2, 1]
[6, 5, 4, 1, 2, 3] [4, 3, 2, 1] [2, 1]
[5, 4, 3, 1, 2] [5, 4, 3, 1, 2] [2, 1]


### making influence grid

In [30]:
influence_locations=[]
for x in pre_dist_location_x_list:
    x=str(x)
    for y in pre_dist_location_y_list:
        y=str(y)
        for z in pre_dist_location_z_list:
            z=str(z)
            influence_x_y_z = []
            influence_x_y_z.append(x)
            influence_x_y_z.append(y)
            influence_x_y_z.append(z)
            influence_location = '_'.join(influence_x_y_z)            
            influence_locations.append(influence_location)

print(influence_locations)


NameError: name 'pre_dist_location_x_list' is not defined

In [55]:
# 자신의 그리드 제거
for location_except, activity in act_grids.items():
    while location_except in influence_locations:
        influence_locations.remove(location_except)
        
print(influence_locations)

['5_5_2', '5_5_1', '5_4_2', '5_4_1', '5_3_2', '5_3_1', '5_1_2', '5_1_1', '5_2_2', '5_2_1', '4_5_2', '4_5_1', '4_4_2', '4_4_1', '4_3_2', '4_3_1', '4_1_2', '4_1_1', '4_2_2', '4_2_1', '3_5_2', '3_5_1', '3_4_2', '3_4_1', '3_3_1', '3_1_1', '3_2_1', '1_5_2', '1_5_1', '1_4_2', '1_4_1', '1_3_2', '1_3_1', '1_1_2', '1_1_1', '1_2_2', '1_2_1', '2_5_2', '2_5_1', '2_4_2', '2_4_1', '2_3_1', '2_1_2', '2_1_1', '2_2_1']


### activity in influence locations 

In [75]:
#location 리스트와 o일의 작업리스트 중 influence location에 있는 거만 작업리스트 만들기
acts_in_Influence_locations = {}
for location, activity in schedule.iterrows():
     if activity["Unnamed: 0"] in influence_locations:
        acts_in_Influence_locations[activity["Unnamed: 0"]]=activity[1:20] #1은 day로 치환 필요
print(acts_in_Influence_locations)


{'2_1_1': 0        NaN
1        NaN
2     S10010
3        NaN
4        NaN
5        NaN
6        NaN
7     S10020
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13    S10030
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
Name: 10, dtype: object, '2_2_1': 0        NaN
1        NaN
2     S10010
3        NaN
4        NaN
5        NaN
6        NaN
7     S10020
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13    S10030
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
Name: 11, dtype: object, '2_3_1': 0        NaN
1     S10010
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7     S10020
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13    S10030
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
Name: 12, dtype: object, '3_1_1': 0        NaN
1     S10010
2        NaN
3        NaN
4        NaN
5        NaN
6     S10020
7        NaN
8        NaN
9        NaN
10       NaN
11  

### checking activity logic with influence locations

In [ ]:
#영향받는 location에 있는 작업의 당일과 이후 20일이내에 check_pre_dist_act(검사하고 있는 작업)보다 선행인지 확인
# 선행작업 있으면 작업을 민다

In [74]:
activity_order = pd.read_excel(navipath.activity_order)
activity_order.head()

,predecessor,successor
0,A41510,A44510
1,A41510,A45510
2,A41510,A46010
3,A41510,A47510
4,A41510,A48510


In [ ]:

def check_activity_order_within_work(local_schedule, activity_code):
    global activity_book

    workday = [day for day in local_schedule.keys() if local_schedule[day] == activity_code][0]

    conflict_items = []
    for day in range(workday, max(local_schedule.keys())+1, 1):
        try:
            activity_code2 = local_schedule[day]
            activity_1 = activity_book[activity_code]
            activity_2 = activity_book[activity_code2]
        except KeyError:
            continue
        
        if activity_code2 in activity_1.successor:
            order = 'fine'
        elif activity_code2 in activity_1.predecessor:
            order = 'conflict'
        else:
            order = 'irrelevant'

        if order == 'conflict':
            conflict_items.append((day, activity_code2))

    return conflict_items

def reorder_activity(local_schedule, activity_code, conflict_items):
    local_schedule_updated = {}

    current = [day for day in local_schedule.keys() if local_schedule[day] == activity_code][0]
    move_to = max([day for day, _ in conflict_items])

    for day in range(0, len(local_schedule), 1):
        if day < current or day > move_to:
            local_schedule_updated[day] = local_schedule[day]
        elif day == current:
            local_schedule_updated[move_to] = local_schedule[day]
        else:
            local_schedule_updated[day-1] = local_schedule[day]

    return {d: a for d, a in sorted(local_schedule_updated.items(), key=lambda x:x[0])}

def update_order_in_local_schedule(local_schedule, verbose=False):
    for day, activity_code in sorted(local_schedule.items(), key=lambda x:x[0]):
        conflict_items = check_activity_order_within_work(local_schedule, activity_code)
        
        if verbose:
            print('{:2}: {}'.format(day, activity_code))
        else:
            pass

        if conflict_items:
            for conflicted_day, conflicted_activity_code in conflict_items:
                print('  | Conflicts at {}({}) <-> {}({})'.format(activity_code, day, conflicted_activity_code, conflicted_day))

            local_schedule_updated = deepcopy(reorder_activity(local_schedule, activity_code, conflict_items))
            return local_schedule_updated
        else:
            continue

    return local_schedule

def update_order(schedule, verbose_iter=False, verbose_local=False):
    schedule_updated = defaultdict(dict)

    iteration = 0
    for location in schedule.keys():
        local_schedule_original = deepcopy(schedule[location])
        while True:
            iteration += 1

            if verbose_iter:
                print('============================================================')
                print('Iteration: {:03,d} ({})'.format(iteration, location))
            else:
                pass

            local_schedule_updated = deepcopy(update_order_in_local_schedule(local_schedule=local_schedule_original, verbose=verbose_local))
            if navifunc.compare_local_schedule(local_schedule_original, local_schedule_updated) == 'same':
                schedule_updated[location] = deepcopy(local_schedule_updated)
                break
            else:
                local_schedule_original = deepcopy(local_schedule_updated)

    return schedule_updated



In [ ]:
# def set_orders_in_activity_book():
#     global fname_activity_book

#     with open(os.path.sep.join((navipath.fdir_component, fname_activity_book)), 'rb') as f:
#         activity_book = pk.load(f)

#     activity_order = pd.read_excel(navipath.activity_order)
#     key_errors = []
#     for _, line in activity_order.iterrows():
#         predecessor_code = line['predecessor']
#         successor_code = line['successor']

#         try:
#             predecessor_activity = deepcopy(activity_book[predecessor_code])
#         except KeyError:
#             key_errors.append(predecessor_code)
#             continue
#         try:
#             successor_activity = deepcopy(activity_book[successor_code])
#         except KeyError:
#             key_errors.append(successor_code)
#             continue

#         predecessor_activity.add_successor(successor_activity.code)
#         successor_activity.add_predecessor(predecessor_activity.code)

#         activity_book[predecessor_code] = predecessor_activity
#         activity_book[successor_code] = successor_activity

#     updates = [True]
#     while any(updates) == True:
#         updates = []
#         for activity_code in activity_book:
#             activity = activity_book[activity_code]

#             existing_preds = deepcopy(list(set(activity.predecessor)))
#             for pred_code in activity.predecessor:
#                 pred_of_pred = list(set(activity_book[pred_code].predecessor))
#                 updated_preds = deepcopy(list(set(existing_preds+pred_of_pred)))
#                 if set(updated_preds) != set(existing_preds):
#                     activity_book[activity_code].predecessor = updated_preds
#                     updates.append(True)
            
#             existing_succs = deepcopy(list(set(activity.successor)))
#             for succ_code in activity.successor:
#                 succ_of_succ = list(set(activity_book[succ_code].successor))
#                 updated_succs = deepcopy(list(set(existing_succs+succ_of_succ)))
#                 if set(updated_succs) != set(existing_succs):
#                     activity_book[activity_code].successor = updated_succs
#                     updates.append(True)

#     with open(os.path.sep.join((navipath.fdir_component, fname_activity_book)), 'wb') as f:
#         pk.dump(activity_book, f)

#     print('============================================================')
#     print('Set order information to ActivityBook')
#     print('  | fdir : {}'.format(navipath.fdir_component))
#     print('  | fname: {}'.format(fname_activity_book))

#     if key_errors:
#         print('Errors on ActivityOrder template')
#         key_errors = list(set(key_errors))
#         for activity_code in key_errors:
#             print('  | Absent in ActivityBook: {}'.format(activity_code))
#     else:
#         pass


In [ ]:
#제 제약조건상에 오류를 발견했어요....
#골조 Workpackage(우리 파일에 Major Actiity)에 세부 작업 먹놓기.... 타설, 양생까지 작업이 있다면
#하부층의 양생이 완료된 후에 상부층 먹놓기가 가능하니까,,, 그냥 선후 작업만 확인해서는 안되네요
#동일층에서는 먹놓기 생산성 만큼 마무리 되면 다른 구역의 먹놓기 작업이 진행가능....
#제약조건이 추가되어야 하네요....
#머리가 좀 아프므로, 레벨의 변화 없이 동일 레벨에서 작업하는거로 데이타 파일을 다시 구성해야겠습니다. 
#1층 골조와 2....층 골조 Code를 다르게해서 선후 조건을 걸어야겠어요.... 고민....

#그리고 대세에 영향이 없을 수 있으므로 
#동일층 내에서 공정계획 자동화하는거에 집중하고
#추가로 생산성 고려할때 가까운 location을 우선배정하는거 까지만 제약 추가하고
#시각화 좀 고민하고
#논문작성에 집중해볼까합니다.
#그래도 희망이 보이네요^^ 감사해요
